In [1]:
import torchvision
from torchvision.datasets import CIFAR10
import torch
torch.manual_seed(0)


In [2]:
from sklearn.model_selection import KFold

In [3]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
class MyDataset(Dataset):
   def __init__(self, np_file_paths):
        #self.data = torch.FloatTensor(data.values.astype('float'))
        self.files = os.listdir(np_file_paths)
        self.np_file_paths = np_file_paths
   def __getitem__(self, index):
     x = np.load("{}/{}".format(self.np_file_paths, self.files[index]))
     x = torch.from_numpy(x).float()
     label = torch.tensor(float(self.files[index].split("_")[4]))
     return x,label
   def __len__(self):
        return len(self.files)


#train_dataset = '/content/gdrive/"My Drive"/trainprob'
#test_dataset = '/content/gdrive/"My Drive"/testprob'


In [7]:
train_dataset =  MyDataset('/content/drive/MyDrive/trainprob2/')
test_dataset = MyDataset('/content/drive/MyDrive/testprob2/')
val_dataset = MyDataset('/content/drive/MyDrive/valueprob2/')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
test_dataset[30] #как слить измерения 4 и 22


(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.]],
 
         [[0., 1., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 1., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
       

In [9]:
print(test_dataset[12])

(tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0.,

In [10]:

trainloader = DataLoader(train_dataset, batch_size = 64, shuffle = True)#в train shuffle нуженб, в  val и test не нужен
valloader = DataLoader(val_dataset, batch_size = 64)
testloader = DataLoader(test_dataset, batch_size = 64)

In [11]:
#у меня 22, а не 20
#сетка имеет один и тот же размер везде, будет плохо работать, количество входных выходных каналов #16 32 64  и обратно
import torch
import torch.nn as nn
class _Model(torch.nn.Module):
    """A neural network model to predict phylogenetic trees."""

    def __init__(self):
        """Create a neural network model."""
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv1d(88, 88, 1, groups=22),
            torch.nn.BatchNorm1d(88),
            torch.nn.ReLU(),
            torch.nn.Conv1d(88, 32, 1),
            torch.nn.BatchNorm1d(32),
            torch.nn.ReLU(),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AdaptiveAvgPool1d(1),
        )
        self.classifier = torch.nn.Linear(32, 1)

    def forward(self, x):
        x = x.view(x.size()[0], 88, -1)
        x = self.conv(x).squeeze(dim=2)
        return self.classifier(x)
class _ResidueModule(torch.nn.Module):

    def __init__(self, channel_count):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv1d(channel_count, channel_count, 1),
            torch.nn.BatchNorm1d(channel_count),
            torch.nn.ReLU(),
            torch.nn.Conv1d(channel_count, channel_count, 1),
            torch.nn.BatchNorm1d(channel_count),
            torch.nn.ReLU()

        )

    def forward(self, x):
        return x + self.layers(x)
# в центре сетка пожирнее, в начале конце поменьше skipped connection

In [12]:
'''
def validate(model,data_loader,device):
  correct = 0
  total = 0
  with torch.no_grad():
    for imgs, labels in data_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        output = model.forward(imgs) # get output
            #print(Y_pred)
        predicted=torch.sigmoid(output).squeeze()
        total += labels.shape[0] # all examples
            #print(labels.shape[0])
            #print(predicted.shape)
        correct += ((predicted>0.5) == labels).sum().item()

  return correct / total

'''

'\ndef validate(model,data_loader,device):\n  correct = 0\n  total = 0\n  with torch.no_grad():\n    for imgs, labels in data_loader:\n        imgs = imgs.to(device)\n        labels = labels.to(device)\n        output = model.forward(imgs) # get output\n            #print(Y_pred)\n        predicted=torch.sigmoid(output).squeeze()\n        total += labels.shape[0] # all examples\n            #print(labels.shape[0])\n            #print(predicted.shape)\n        correct += ((predicted>0.5) == labels).sum().item()\n  \n  return correct / total  \n\n'

In [13]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn import metrics
def calaculate_accuracy(model, data_loader):
    correct, total = 0, 0
    f1 = []
    with torch.no_grad():
        for batch in data_loader: # get batch
            imgs, labels = batch # parse batch
            imgs, labels = imgs.to(device), labels.to(device)
            Y_pred = model.forward(imgs) # get output
            predicted=torch.sigmoid(Y_pred).squeeze()
            total += labels.shape[0] # all examples
            labelcont = []
            predictedcont = []
            for i in range(len(labels)):#лучше int
              labelcont.append(float(labels[i]))
            for j in range(len(predicted)):#вынести выше,один classification report
              if predicted[j] > 0.5:
                predictedcont.append(float(1))
              else:
                predictedcont.append(float(0))
            correct += ((predicted>0.5) == labels).sum().item()
            print(torch.tensor(labelcont))
            print(torch.tensor(predictedcont))
            print(classification_report(torch.tensor(labelcont), torch.tensor(predictedcont)))
            f1.append(f1_score(labelcont, predictedcont, average='binary'))
    plt.figure(figsize=(8, 6))
    plt.title("F1")

    plt.plot(f1)  #f1 номер батча
    return correct / total

In [ ]:
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # change run time to gpu to fast training

model = _Model().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 8
loss_histtr = [] # for plotting
loss_histval = []
best_accuracyval = 0
best_accuracytr = 0
lossacctr = []
lossaccval = []
for epoch in range(num_epochs):
    model.train()
    hist_loss = 0
    hist_loss2 = 0
    for _, batch in enumerate(trainloader, 0): # get batch
        # parse batch
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        # sets the gradients of all optimized tensors to zero.
        optimizer.zero_grad()
        # get outputs
        Y_pred = model(imgs)
        Y_pred = Y_pred.squeeze(axis=1)
        # calculate loss
        loss = criterion(Y_pred, labels)
        # calculate gradients
        loss.backward()
        # performs a single optimization step (parameter update)
        optimizer.step()
        hist_loss += loss.item()

    accuracytr = calaculate_accuracy(model,trainloader)
    if best_accuracytr < accuracytr:
      best_accuracytr = accuracytr
    loss_histtr.append(hist_loss /len(trainloader))
    lossacctr.append(accuracytr)
    print(f"Epoch={epoch}  train_loss={loss_histtr[epoch]:.4f} accuracytr = {accuracytr:.4f}")
    model.eval()
    for _, batch2 in enumerate(valloader, 0): # get batch
        # parse batch
        imgs2, labels2 = batch2
        imgs2, labels2 = imgs2.to(device), labels2.to(device)
        # sets the gradients of all optimized tensors to zero.
        # get outputs
        Y_pred2 = model(imgs2)
        Y_pred2 = Y_pred2.squeeze(axis=1)
        loss2 = criterion(Y_pred2, labels2)
        hist_loss2 += loss2.item()
    accuracyval = calaculate_accuracy(model,valloader)
    if best_accuracyval < accuracyval:
      best_accuracyval = accuracyval
    lossaccval.append(accuracyval)
    loss_histval.append(hist_loss2 /len(valloader))
    print(f"Epoch={epoch}  val_loss={loss_histval[epoch]:.4f} accuracyval = {accuracyval:.4f}")

tensor([1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
        1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        1., 1., 0., 1., 0., 1., 1., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
              precision    recall  f1-score   support

         0.0       0.66      0.95      0.78        40
         1.0       0.67      0.17      0.27        24

    accuracy                           0.66        64
   macro avg       0.66      0.56      0.52        64
weighted avg       0.66      0.66      0.58        64

tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0.,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tensor([0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1.,
        1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.,
        1., 1., 1., 1., 0., 0., 1., 1., 0., 1.])
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
              precision    recall  f1-score   support

         0.0       0.60      0.97      0.74        38
         1.0       0.50      0.04      0.07        26

    accuracy                           0.59        64
   macro avg       0.55      0.51      0.41        64
weighted avg       0.56      0.59      0.47        64

tensor([0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
        0., 0.,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tensor([1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
        0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1.,
        0., 1., 1., 1., 0., 0., 1., 0., 1., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
              precision    recall  f1-score   support

         0.0       0.57      1.00      0.73        35
         1.0       1.00      0.10      0.19        29

    accuracy                           0.59        64
   macro avg       0.79      0.55      0.46        64
weighted avg       0.77      0.59      0.48        64

tensor([1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
        0., 0.,

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1.,
        0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
              precision    recall  f1-score   support

         0.0       0.76      1.00      0.86        47
         1.0       1.00      0.12      0.21        17

    accuracy                           0.77        64
   macro avg       0.88      0.56      0.54        64
weighted avg       0.82      0.77      0.69        64

tensor([1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
        0., 1.,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 1., 0., 0., 0., 0., 0., 0., 1.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
              precision    recall  f1-score   support

         0.0       0.78      0.98      0.87        50
         1.0       0.00      0.00      0.00        14

    accuracy                           0.77        64
   macro avg       0.39      0.49      0.43        64
weighted avg       0.61      0.77      0.68        64

tensor([0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
        0., 0.,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.title("Loss model 3")
plt.plot(loss_histval,label = "val")
plt.plot(loss_histtr, label = "train")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.title("Accuracy model 3")
plt.plot(lossaccval,label = "val")
plt.plot(lossacctr,label = "train")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show()

In [ ]:

acc_train = round(calaculate_accuracy(model, trainloader), 3)#метрики по обеим классам отдельно classification report знает, что два класса
print(f"Accuracy train = {acc_train}")
acc_val = round(calaculate_accuracy(model, valloader), 3)
print(f"Accuracy val = {acc_val}")
acc_test = round(calaculate_accuracy(model, testloader), 3)
print(f"Accuracy test = {acc_test}")
